In [2]:
pip install torch torchvision transformers

     |████████████████████████████████| 542kB 2.8MB/s 
     |████████████████████████████████| 870kB 45.9MB/s 
     |████████████████████████████████| 1.0MB 33.9MB/s 
     |████████████████████████████████| 3.7MB 36.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=88d5db78e5c7e6c7a3f33f35e96e2e7b6a78a9fcdb4102655bd43dcb07b05a4a
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [3]:
!git clone https://github.com/huggingface/transformers \
&& cd transformers 
# && git checkout a3085020ed0d81d4903c50967687192e3101e770 

Cloning into 'transformers'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 23132 (delta 121), reused 169 (delta 103), pack-reused 22929
Receiving objects: 100% (23132/23132), 13.66 MiB | 22.17 MiB/s, done.
Resolving deltas: 100% (16563/16563), done.


In [4]:
!git clone https://github.com/nyu-mll/GLUE-baselines.git 
!git clone https://github.com/wasiahmad/paraphrase_identification.git \
&& python GLUE-baselines/download_glue_data.py --data_dir glue_data --tasks all --path_to_mrpc=paraphrase_identification/dataset/msr-paraphrase-corpus

Cloning into 'GLUE-baselines'...
remote: Enumerating objects: 886, done.
remote: Total 886 (delta 0), reused 0 (delta 0), pack-reused 886
Receiving objects: 100% (886/886), 1.47 MiB | 15.41 MiB/s, done.
Resolving deltas: 100% (609/609), done.
Cloning into 'paraphrase_identification'...
remote: Enumerating objects: 68, done.
remote: Total 68 (delta 0), reused 0 (delta 0), pack-reused 68
Unpacking objects: 100% (68/68), done.
	Completed!
	Completed!
Processing MRPC...
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!


In [5]:
!mkdir dataset \
&& cd dataset \
&& wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json \
&& wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2020-03-26 15:55:21--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.111.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   148MB/s    in 0.3s    

2020-03-26 15:55:22 (148 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2020-03-26 15:55:22--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.111.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

In [6]:
## Mount Drive into Colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
!ls
!ls drive/'My Drive'/Layer\ 1\ Models

dataset  GLUE-baselines  paraphrase_identification  transformers
drive	 glue_data	 sample_data
Final_Test_Set.csv
roberta_for_section_Cell_Division_model.pth
roberta_for_section_Cell_Strucuture_and_transport_model.pth
roberta_for_section_Communicable_diseases_model.pth
roberta_for_section_Genetics_and_evolution_model.pth
roberta_for_section_Non-Communicable_diseases_model.pth
roberta_for_section_Organisation_and_the_digestive_system_model.pth
roberta_for_section_Organising_Animals_and_Plants_model.pth
roberta_for_section_Photosynthesis_model.pth
roberta_for_section_Preventing_and_treating_disease_model.pth
roberta_for_section_Reproduction_model.pth
roberta_for_section_Respiration_model.pth
roberta_for_section_The_human_nervous_system_model.pth
roberta_full_subsections_model.pth
roberta_main_model.pth
roberta_sections_model.pth
test_dataset.csv
train_dataset.csv


In [8]:
import pandas as pd
import numpy as np
import codecs
import re
from collections import Counter
from matplotlib import pyplot as plt
from operator import itemgetter
from tqdm import tqdm_notebook

import nltk
from nltk.corpus import stopwords
import string
from nltk.stem.porter import PorterStemmer 
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
nltk.download('punkt')

## Torch Modules
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler


import os
import time

from transformers import (
    RobertaConfig, 
    RobertaModel,
    RobertaTokenizer,
    RobertaForSequenceClassification,
    AlbertConfig,
    AlbertForQuestionAnswering,
    AlbertTokenizer,
    squad_convert_examples_to_features
)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
## Check if Cuda is Available
print(torch.cuda.is_available())

True


In [0]:
dataset_url = 'https://raw.githubusercontent.com/DhenPadilla/KSQuAD/master/Bio_GCSE.csv'
df = pd.read_csv(dataset_url)
passage_df = df

In [11]:
passage_df.head(4)

,Main category,Section,Sub-section,Text,Questions
0,Cells and Organisms,Cell Strucuture and transport,Microscopes,"Living things are all made up of cells, but mo...",Name one advantage and one disadvantage of a l...
1,Cells and Organisms,Cell Strucuture and transport,Animal and plant cells,The cells that make up your body are typical a...,Name the main structures you would expect to f...
2,Cells and Organisms,Cell Strucuture and transport,Eukaryotic and prokaryotic cells,Animal and plant cells are example’s of eukary...,Describe what flagella are ? \nName one use of...
3,Cells and Organisms,Cell Strucuture and transport,Specialisation in animal cells,Although the smallest living organisms are onl...,Name one adaption for the specialised nerver c...


In [0]:
def remove_nulls(dataset, print_out=True):
  dataset['question'].replace('', np.nan, inplace=True)
  dataset.dropna(subset=['question'], inplace=True)
  dataset.reset_index(drop=True, inplace=True)
  for label in dataset["label"].unique():
    if(print_out):
      print("Class: {}, # questions: {}".format(label, dataset.label.value_counts()[label]))
  return dataset

In [13]:
# MAIN CATEGORY DATA SET
category_dataset = pd.DataFrame(columns = ['question', 'label'])

for idx, passage in enumerate(passage_df["Questions"][:]):
  for question in passage.split("\n"):
    small_frame = pd.Series({"question": question, "label": passage_df["Main category"][idx]}) 
    category_dataset = category_dataset.append(small_frame, ignore_index=True)
remove_nulls(category_dataset)

print(category_dataset["label"].size)

Class: Cells and Organisms, # questions: 533
Class: Disease and bioenergetics, # questions: 320
Class: Biological responses, # questions: 80
Class: Genetics and reproduction, # questions: 209
1142


In [14]:
# MAIN CATEGORY -> SECTION DATA SET
section_dataset = pd.DataFrame(columns = ['question', 'label'])

for idx, passage in enumerate(passage_df["Questions"][:]):
  for question in passage.split("\n"):
    small_frame = pd.Series({"question": question, "label": passage_df["Section"][idx]}) 
    section_dataset = section_dataset.append(small_frame, ignore_index=True)
remove_nulls(section_dataset)

print(section_dataset["label"].size)

Class: Cell Strucuture and transport, # questions: 273
Class: Cell Division, # questions: 63
Class: Organisation and the digestive system, # questions: 84
Class: Organising Animals and Plants, # questions: 113
Class: Communicable diseases, # questions: 112
Class: Preventing and treating disease, # questions: 68
Class: Non-Communicable diseases, # questions: 73
Class: Photosynthesis, # questions: 26
Class: Respiration, # questions: 41
Class: The human nervous system, # questions: 80
Class: Reproduction, # questions: 76
Class: Genetics and evolution, # questions: 133
1142


In [15]:
# OVERALL SUB-SECTION DATA SET
full_subsection_dataset = pd.DataFrame(columns = ['question', 'label'])

for idx, passage in enumerate(passage_df["Questions"][:]):
  for question in passage.split("\n"):
    small_frame = pd.Series({"question": question, "label": passage_df["Sub-section"][idx]}) 
    full_subsection_dataset = full_subsection_dataset.append(small_frame, ignore_index=True)
remove_nulls(full_subsection_dataset)

print(full_subsection_dataset["label"].size)
print(full_subsection_dataset["question"][23])

Class: Microscopes, # questions: 24
Class: Animal and plant cells, # questions: 41
Class: Eukaryotic and prokaryotic cells, # questions: 35
Class: Specialisation in animal cells, # questions: 52
Class: Specialisation in plant cells, # questions: 34
Class: Diffusion, # questions: 30
Class: Osmosis, # questions: 12
Class: Osmosis in plants, # questions: 14
Class: Active transport, # questions: 16
Class: Exchanging materials, # questions: 15
Class: Cell division, # questions: 24
Class: Growth and differentiation, # questions: 17
Class: Stem cells, # questions: 20
Class: Stem cell dilemmas, # questions: 2
Class: Tissues and organs, # questions: 11
Class: The human digestive system, # questions: 12
Class: The chemistry of food, # questions: 16
Class: Catalysts and enzymes, # questions: 7
Class: Factors affecting enzyme action, # questions: 10
Class: How the digestive system works, # questions: 15
Class: Making digestion efficient, # questions: 13
Class: The blood, # questions: 15
Class: The

In [0]:
# SECTION -> SUBSECTION DATA SETS (Dictionary)
subsection_datasets = {}

for section in passage_df["Section"].unique():
  dataset = pd.DataFrame(columns = ['question', 'label'])
  section_df = passage_df.loc[passage_df["Section"] == section]
  for idx, row in section_df.iterrows():
    for question in row["Questions"].split("\n"):
      small_frame = pd.Series({"question": question, "label": section_df["Sub-section"][idx]}) 
      dataset = dataset.append(small_frame, ignore_index=True)
      remove_nulls(dataset, print_out=False)
      subsection_datasets[section] = dataset

In [17]:
for section, df in subsection_datasets.items():
  print("Class: {}, # questions: {}".format(section, df["label"].size))

Class: Cell Strucuture and transport, # questions: 273
Class: Cell Division, # questions: 63
Class: Organisation and the digestive system, # questions: 84
Class: Organising Animals and Plants, # questions: 113
Class: Communicable diseases, # questions: 112
Class: Preventing and treating disease, # questions: 68
Class: Non-Communicable diseases, # questions: 73
Class: Photosynthesis, # questions: 26
Class: Respiration, # questions: 41
Class: The human nervous system, # questions: 80
Class: Reproduction, # questions: 76
Class: Genetics and evolution, # questions: 133


In [0]:
def label_to_index(dataset):
  label_to_ix = {}
  for label in dataset.label:
        if label not in label_to_ix:
            label_to_ix[label]=len(label_to_ix)
  return label_to_ix

In [19]:
# Main Category Labels
main_category_labels = label_to_index(category_dataset)
print(main_category_labels)

{'Cells and Organisms': 0, 'Disease and bioenergetics': 1, 'Biological responses': 2, 'Genetics and reproduction': 3}


In [20]:
# Section labels
section_labels = label_to_index(section_dataset)
print(section_labels)

{'Cell Strucuture and transport': 0, 'Cell Division': 1, 'Organisation and the digestive system': 2, 'Organising Animals and Plants': 3, 'Communicable diseases': 4, 'Preventing and treating disease': 5, 'Non-Communicable diseases': 6, 'Photosynthesis': 7, 'Respiration': 8, 'The human nervous system': 9, 'Reproduction': 10, 'Genetics and evolution': 11}


In [21]:
# Sub-section labels (overall)
full_subsection_labels = label_to_index(full_subsection_dataset)
print(full_subsection_labels)

{'Microscopes': 0, 'Animal and plant cells': 1, 'Eukaryotic and prokaryotic cells': 2, 'Specialisation in animal cells': 3, 'Specialisation in plant cells': 4, 'Diffusion': 5, 'Osmosis': 6, 'Osmosis in plants': 7, 'Active transport': 8, 'Exchanging materials': 9, 'Cell division': 10, 'Growth and differentiation': 11, 'Stem cells': 12, 'Stem cell dilemmas': 13, 'Tissues and organs': 14, 'The human digestive system': 15, 'The chemistry of food': 16, 'Catalysts and enzymes': 17, 'Factors affecting enzyme action': 18, 'How the digestive system works': 19, 'Making digestion efficient': 20, 'The blood': 21, 'The blood vessels': 22, 'The heart': 23, 'Helping the heart': 24, 'Breathing and gas exchange': 25, 'Tissues and organs in plants': 26, 'Transport systems in plants': 27, 'Evaporation and transpiration': 28, 'Factors affecting transpiration': 29, 'Health and disease': 30, 'Pathogens and disease': 31, 'growing Bacteria in the lab': 32, 'Preventing bacteria growth': 33, 'Preventing infecti

In [22]:
# Sub-Section labels

subsection_labels = {}

for section, df in subsection_datasets.items():
  labels = label_to_index(df)
  subsection_labels[section] = labels
  print(section)
  print(labels)
  print("")

# NOTE: Have to reinitialise subsection_labels variable later bc it causes an error

Cell Strucuture and transport
{'Microscopes': 0, 'Animal and plant cells': 1, 'Eukaryotic and prokaryotic cells': 2, 'Specialisation in animal cells': 3, 'Specialisation in plant cells': 4, 'Diffusion': 5, 'Osmosis': 6, 'Osmosis in plants': 7, 'Active transport': 8, 'Exchanging materials': 9}

Cell Division
{'Cell division': 0, 'Growth and differentiation': 1, 'Stem cells': 2, 'Stem cell dilemmas': 3}

Organisation and the digestive system
{'Tissues and organs': 0, 'The human digestive system': 1, 'The chemistry of food': 2, 'Catalysts and enzymes': 3, 'Factors affecting enzyme action': 4, 'How the digestive system works': 5, 'Making digestion efficient': 6}

Organising Animals and Plants
{'The blood': 0, 'The blood vessels': 1, 'The heart': 2, 'Helping the heart': 3, 'Breathing and gas exchange': 4, 'Tissues and organs in plants': 5, 'Transport systems in plants': 6, 'Evaporation and transpiration': 7, 'Factors affecting transpiration': 8}

Communicable diseases
{'Health and disease':

### Configure a RoBERTa model to be used to train our dataset

In [23]:
# Create RoBERTa model method for multiple instances
def create_roberta_model(labels):
  config = RobertaConfig.from_pretrained('roberta-base')
  config.num_labels = len(list(labels.values()))
  model = RobertaForSequenceClassification(config)
  return model

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [24]:
# For Main-Category classification:
main_cat_model = create_roberta_model(main_category_labels)

# For Section classification:
section_model = create_roberta_model(section_labels)

# For FULL Section classification:
full_subsection_model = create_roberta_model(full_subsection_labels)

# For Sub section classifcations:
subsection_models = {}
for section, subsection_labels in subsection_labels.items():
  print("Creating RoBERTa for: ", section)
  subsection_models[section] = create_roberta_model(subsection_labels)


Creating RoBERTa for:  Cell Strucuture and transport
Creating RoBERTa for:  Cell Division
Creating RoBERTa for:  Organisation and the digestive system
Creating RoBERTa for:  Organising Animals and Plants
Creating RoBERTa for:  Communicable diseases
Creating RoBERTa for:  Preventing and treating disease
Creating RoBERTa for:  Non-Communicable diseases
Creating RoBERTa for:  Photosynthesis
Creating RoBERTa for:  Respiration
Creating RoBERTa for:  The human nervous system
Creating RoBERTa for:  Reproduction
Creating RoBERTa for:  Genetics and evolution


### The following set of functions are helper-functions to create RoBERTa inputs from (question, label) pairs

In [0]:
def feature_preparation(seq, max_length = 300, zero_pad = False, include_CLS_token = True, include_SEP_token = True):
    ## Tokenzine Input
    tokens_a = tokenizer.tokenize(seq)

    ## Truncate
    if len(tokens_a) > max_length - 2:
        tokens_a = tokens_a[0:(max_length - 2)]

    ## Initialize Tokens
    tokens = []
    if include_CLS_token:
        tokens.append(tokenizer.cls_token)

    ## Add Tokens and separators
    for token in tokens_a:
        tokens.append(token)

    if include_SEP_token:
        tokens.append(tokenizer.sep_token)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    ## Input Mask 
    input_mask = [1] * len(input_ids)

    ## Zero-pad sequence lenght
    if zero_pad:
        while len(input_ids) < max_length:
            input_ids.append(0)
            input_mask.append(0)
    return torch.tensor(input_ids).unsqueeze(0), input_mask

In [26]:
# Check question preparation if correct for each dataset:
cat_question = category_dataset['question'][50]
print("Main Category")
print(cat_question)
print(feature_preparation(cat_question))
print("")

sec_question = section_dataset['question'][23]
print("Section")
print(sec_question)
print(feature_preparation(sec_question))
print("")

full_subsec_question = full_subsection_dataset['question'][23]
print("Full Subsection")
print(full_subsec_question)
print(feature_preparation(full_subsec_question))
print("")

subsec_question = subsection_datasets["Cell Division"]['question'][9]
print("Subsection: Cell Division")
print(subsec_question)
print(feature_preparation(subsec_question))
print("")

Main Category
What are algae?
(tensor([[    0,  2264,    32, 25237,   116,     2]]), [1, 1, 1, 1, 1, 1])

Section
What is the resolving power of an electron microscope?
(tensor([[    0,  2264,    16,     5, 20006,   476,     9,    41, 35235, 32518,
           116,     2]]), [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Full Subsection
What is the resolving power of an electron microscope?
(tensor([[    0,  2264,    16,     5, 20006,   476,     9,    41, 35235, 32518,
           116,     2]]), [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Subsection: Cell Division
How common is measles in the UK?
(tensor([[    0,  6179,  1537,    16, 26143,    11,     5,   987,   116,     2]]), [1, 1, 1, 1, 1, 1, 1, 1, 1, 1])



In [0]:
class prepare_input(Dataset):
    def __init__(self, dataframe, mode="", section_to_prepare=""):
        self.len = len(dataframe)
        self.data = dataframe
        self.mode = mode
        self.section_to_prepare = section_to_prepare
        
    def __getitem__(self, index):
        question = self.data.question[index]
        label = self.data.label[index]
        X, _  = feature_preparation(question)
        if(self.section_to_prepare and self.mode == "SUBSECTIONS"):
          y = subsection_labels[self.section_to_prepare][self.data.label[index]]
        elif(self.mode == "FULL_SUBSECTIONS"):
          y = full_subsection_labels[self.data.label[index]]
        elif(self.mode == "SECTIONS"):
          y = section_labels[self.data.label[index]]
        else:
          y = main_category_labels[self.data.label[index]]
        return X, y
    
    def __len__(self):
        return self.len

### Create train-test split (80:20)

In [0]:
train_size = 0.80

In [0]:
def find_section_for(subsection):
  return passage_df.loc[passage_df["Sub-section"] == subsection , 'Section'].values[0]

def find_category_for(subsection):
  return passage_df.loc[passage_df["Sub-section"] == subsection , 'Main category'].values[0]

In [30]:
# For Sub-sections:
subsection_datasets_train = {}
subsection_datasets_test = {}

# Main Category train-test split
category_train_dataset = pd.DataFrame(columns = ['question', 'label'])
category_test_dataset  = pd.DataFrame(columns = ['question', 'label'])

# Full Sub-section train-test
full_subsection_train_dataset = pd.DataFrame(columns = ['question', 'label'])
full_subsection_test_dataset  = pd.DataFrame(columns = ['question', 'label'])

# Section train-test
section_train_dataset = pd.DataFrame(columns = ['question', 'label'])
section_test_dataset  = pd.DataFrame(columns = ['question', 'label'])

for section, df in subsection_datasets.items():
  train_dataset = df.sample(frac=train_size, random_state=200).reset_index(drop=True)
  test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
  subsection_datasets_train[section] = train_dataset
  subsection_datasets_test[section] = test_dataset

  # Append to Main Category train-test dataset
  category_train_dataset = category_train_dataset.append(train_dataset, ignore_index=True)
  category_test_dataset = category_test_dataset.append(test_dataset, ignore_index=True)
  # Append to section train-test dataset
  section_train_dataset = section_train_dataset.append(train_dataset, ignore_index=True)
  section_test_dataset = section_test_dataset.append(test_dataset, ignore_index=True)
  # Append to full subsection train-test dataset
  full_subsection_train_dataset = full_subsection_train_dataset.append(train_dataset, ignore_index=True)
  full_subsection_test_dataset = full_subsection_test_dataset.append(test_dataset, ignore_index=True)


  print("Section name: ", section)
  print("TRAIN Dataset: {}".format(subsection_datasets_train[section].shape))
  print("TEST Dataset: {}".format(subsection_datasets_test[section].shape))


# Create Main Category train-test split
for _, row in category_train_dataset.iterrows():
  row.label = find_category_for(row.label)

for _, row in category_test_dataset.iterrows():
  row.label = find_category_for(row.label)

# Create section train-test split
for _, row in section_train_dataset.iterrows():
  row.label = find_section_for(row.label)

for _, row in section_test_dataset.iterrows():
  row.label = find_section_for(row.label)

Section name:  Cell Strucuture and transport
TRAIN Dataset: (218, 2)
TEST Dataset: (55, 2)
Section name:  Cell Division
TRAIN Dataset: (50, 2)
TEST Dataset: (13, 2)
Section name:  Organisation and the digestive system
TRAIN Dataset: (67, 2)
TEST Dataset: (17, 2)
Section name:  Organising Animals and Plants
TRAIN Dataset: (90, 2)
TEST Dataset: (23, 2)
Section name:  Communicable diseases
TRAIN Dataset: (90, 2)
TEST Dataset: (22, 2)
Section name:  Preventing and treating disease
TRAIN Dataset: (54, 2)
TEST Dataset: (14, 2)
Section name:  Non-Communicable diseases
TRAIN Dataset: (58, 2)
TEST Dataset: (15, 2)
Section name:  Photosynthesis
TRAIN Dataset: (21, 2)
TEST Dataset: (5, 2)
Section name:  Respiration
TRAIN Dataset: (33, 2)
TEST Dataset: (8, 2)
Section name:  The human nervous system
TRAIN Dataset: (64, 2)
TEST Dataset: (16, 2)
Section name:  Reproduction
TRAIN Dataset: (61, 2)
TEST Dataset: (15, 2)
Section name:  Genetics and evolution
TRAIN Dataset: (106, 2)
TEST Dataset: (27, 2)


In [31]:
print("TRAIN Category Dataset: {}".format(category_train_dataset.shape))
print("TEST Category Dataset: {}".format(category_test_dataset.shape))
print("")
print("TRAIN Section Dataset: {}".format(section_train_dataset.shape))
print("TEST Section Dataset: {}".format(section_test_dataset.shape))
print("")
print("TRAIN Full Sub-section Dataset Size: {}".format(full_subsection_train_dataset.shape))
print("TEST Full Sub-section Dataset Size: {}".format(full_subsection_test_dataset.shape))
print("")
print("No. Unique sections in TRAIN Section Dataset: {}".format(len(section_train_dataset.label.unique())))
print("No. Unique sections in TEST Section Dataset: {}".format(len(section_test_dataset.label.unique())))
print("")
print("No. sections in TRAIN Full Sub-Section Dataset: {}".format(len(full_subsection_train_dataset.label.unique())))
print("No. sections in TEST Full Sub-Section Dataset: {}".format(len(full_subsection_test_dataset.label.unique())))

TRAIN Category Dataset: (912, 2)
TEST Category Dataset: (230, 2)

TRAIN Section Dataset: (912, 2)
TEST Section Dataset: (230, 2)

TRAIN Full Sub-section Dataset Size: (912, 2)
TEST Full Sub-section Dataset Size: (230, 2)

No. Unique sections in TRAIN Section Dataset: 12
No. Unique sections in TEST Section Dataset: 12

No. sections in TRAIN Full Sub-Section Dataset: 75
No. sections in TEST Full Sub-Section Dataset: 22


In [0]:
# Main Category tokenised
category_train_set = prepare_input(category_train_dataset)
category_test_set = prepare_input(category_test_dataset)

In [0]:
# Sections tokenised
section_train_set = prepare_input(section_train_dataset, "SECTIONS")
section_test_set = prepare_input(section_test_dataset, "SECTIONS")

In [0]:
# Subsections tokenised
full_subsection_train_set = prepare_input(full_subsection_train_dataset, "FULL_SUBSECTIONS")
full_subsection_test_set = prepare_input(full_subsection_test_dataset, "FULL_SUBSECTIONS")

In [0]:
subsection_labels = {}

for section, df in subsection_datasets.items():
  labels = label_to_index(df)
  subsection_labels[section] = labels

In [0]:
# Subsections tokenised
subsections_train_set = {}
subsections_test_set = {}

for section, df in subsection_datasets_train.items():
  subsections_train_set[section] = prepare_input(df, "SUBSECTIONS", section)

for section, df in subsection_datasets_test.items():
  subsections_test_set[section] = prepare_input(df, "SUBSECTIONS", section)

### If CUDA's available, config model to use it.

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
main_cat_model = main_cat_model.cuda()
section_model = section_model.cuda()
full_subsection_model = full_subsection_model.cuda()
for model in subsection_models.values():
  model = model.cuda()

In [38]:
#Check if cuda is used for data: 
print("Main Categories:")
print(category_train_set.__getitem__(0)[0].cuda())
print("")

print("Sections:")
print(section_train_set.__getitem__(0)[0].cuda())
print("")

print("Full Sections:")
print(full_subsection_train_set.__getitem__(0)[0].cuda())
print("")

print("Sub-Sections ({}):".format(list(subsections_train_set.keys())[0]))
print(subsections_train_set[list(subsections_train_set.keys())[0]].__getitem__(0)[0].cuda())

Main Categories:
tensor([[    0,  1779,   473, 44937,  1369,  1335,   116,     2]],
       device='cuda:0')

Sections:
tensor([[    0,  1779,   473, 44937,  1369,  1335,   116,     2]],
       device='cuda:0')

Full Sections:
tensor([[    0,  1779,   473, 44937,  1369,  1335,   116,     2]],
       device='cuda:0')

Sub-Sections (Cell Strucuture and transport):
tensor([[    0,  1779,   473, 44937,  1369,  1335,   116,     2]],
       device='cuda:0')


In [0]:
# Config Parameters for training the RoBERTa models
params = {'batch_size': 1,
          'shuffle': True,
          'drop_last': False,
          'num_workers': 1}

# Create the data loaders for each dataset

# CATEGORIES:
category_training_loader = DataLoader(category_train_set, **params)
category_testing_loader = DataLoader(category_test_set, **params)

# SECTIONS:
section_training_loader = DataLoader(section_train_set, **params)
section_testing_loader = DataLoader(section_test_set, **params)

# FULL_SUBSECTIONS:
full_subsection_training_loader = DataLoader(full_subsection_train_set, **params)
full_subsection_testing_loader = DataLoader(full_subsection_test_set, **params)

# SUBSECTIONS:
subsection_training_loaders = {}
for section in subsections_train_set.keys():
  training_loader = DataLoader(subsections_train_set[section], **params)
  subsection_training_loaders[section] = training_loader
subsection_testing_loaders = {}
for section in subsections_test_set.keys():
  test_loader = DataLoader(subsections_test_set[section], **params)
  subsection_testing_loaders[section] = test_loader

## Dictionaries for the models, training & test loaders

In [0]:
model_dict = {"Main" : main_cat_model,
              "Sections" : section_model,
              "Full Sub-sections" : full_subsection_model,
              "Subsections" : subsection_models}

In [0]:
training_loaders = {"Main" : category_training_loader,
                    "Sections" : section_training_loader,
                    "Full Sub-sections" : full_subsection_training_loader,
                    "Subsections" : subsection_training_loaders }

testing_loaders = {"Main" : category_testing_loader,
                    "Sections" : section_testing_loader,
                    "Full Sub-sections" : full_subsection_testing_loader,
                    "Subsections" : subsection_testing_loaders }

In [42]:
## Test a single forward pass 
## for each training set (Category, Section, Sub-section)

# Category
cat_inp  = category_train_set.__getitem__(0)[0].cuda()
cat_output = model_dict["Main"].forward(cat_inp)
print(cat_output)
print(torch.max(cat_output[0], 1))

# Section
sec_inp  = section_train_set .__getitem__(0)[0].cuda()
sec_output = model_dict["Sections"].forward(sec_inp)
print(sec_output)
print(torch.max(sec_output[0], 1))

# Full Sub-sections
full_subsec_inp  = full_subsection_train_set .__getitem__(0)[0].cuda()
full_subsec_output = model_dict["Full Sub-sections"].forward(full_subsec_inp)
print(full_subsec_output)
print(torch.max(full_subsec_output[0], 1))

# Sub-Section (Cell Division)
for section, model in model_dict["Subsections"].items():
  ssec_inp = subsections_train_set[section].__getitem__(0)[0].cuda()
  ssec_output = model.forward(ssec_inp)
  print(ssec_output)
  print(torch.max(ssec_output[0], 1))

(tensor([[ 0.6238, -0.0310,  0.2262, -0.1502]], device='cuda:0',
       grad_fn=<AddmmBackward>),)
torch.return_types.max(
values=tensor([0.6238], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([0], device='cuda:0'))
(tensor([[ 0.5010, -0.6548,  0.3190,  0.2510,  0.3139,  0.3229,  0.2989,  0.0527,
         -0.2255, -0.1279, -0.0752, -0.1801]], device='cuda:0',
       grad_fn=<AddmmBackward>),)
torch.return_types.max(
values=tensor([0.5010], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([0], device='cuda:0'))
(tensor([[ 0.4558,  0.1557,  0.6323,  0.1441, -0.3280,  0.2204,  0.2699,  0.3990,
         -0.2081, -0.1979, -0.2734,  0.3461,  0.2270,  0.1053,  0.3749, -0.3030,
         -0.0872, -0.4990, -0.2190, -0.3069,  0.2123, -0.4515, -0.4414, -0.4560,
         -0.1810, -0.2141,  0.2490, -0.2210, -0.0137, -0.0179,  0.1031,  0.3807,
          0.0285, -0.0185, -0.0081, -0.0917,  0.2773,  0.0456, -0.2553,  0.6711,
          0.2197, -0.1602,  0.0727, -0.1859, -0.5785, -0.09

In [0]:
def train_on(model, training_loader, testing_loader, optimizer, loss_function=nn.CrossEntropyLoss()):
  max_epochs = 3

  print_at = 100 if (len(training_loader.dataset) > 300) else 50

  model = model.train()

  for epoch in tqdm_notebook(range(max_epochs)):
      print("EPOCH -- {}".format(epoch))
      for i, (question, label) in enumerate(training_loader):
          optimizer.zero_grad()
          question = question.squeeze(0)
          if torch.cuda.is_available():
            question = question.cuda()
            label = label.cuda()
          output = model.forward(question)[0]
          _, predicted = torch.max(output, 1)
          
          loss = loss_function(output, label)
          loss.backward()
          optimizer.step()
          
          if i % print_at == 0:
              correct = 0
              total = 0
              for question, label in testing_loader:
                  question = question.squeeze(0)
                  if torch.cuda.is_available():
                    question = question.cuda()
                    label = label.cuda()
                  output = model.forward(question)[0]
                  _, predicted = torch.max(output.data, 1)
                  total += label.size(0)
                  correct += (predicted.cpu() == label.cpu()).sum()
              accuracy = 100.00 * correct.numpy() / total
              print('Iteration: {}. Loss: {}. Accuracy: {}%'.format(i, loss.item(), accuracy))

In [57]:
# Train Main Category Model
# loss_function = nn.CrossEntropyLoss()
learning_rate = 1e-05
category_model_optimizer = optim.Adam(params =  model_dict["Main"].parameters(), lr=learning_rate)
train_on(model_dict["Main"], training_loaders["Main"], testing_loaders["Main"], category_model_optimizer)

# Save and clear model
torch.save(model_dict["Main"].state_dict(), 'drive/My Drive/Layer 1 Models/roberta_main_model.pth')
# model_dict["Main"] = None

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


EPOCH -- 0
Iteration: 0. Loss: 0.6648582220077515. Accuracy: 48.987854251012145%
Iteration: 100. Loss: 3.050795793533325. Accuracy: 48.987854251012145%
Iteration: 200. Loss: 0.8833415508270264. Accuracy: 63.56275303643725%
Iteration: 300. Loss: 0.8599144816398621. Accuracy: 43.31983805668016%
Iteration: 400. Loss: 1.252485990524292. Accuracy: 65.18218623481782%
Iteration: 500. Loss: 2.651005268096924. Accuracy: 62.34817813765182%
Iteration: 600. Loss: 2.418715000152588. Accuracy: 35.22267206477733%
Iteration: 700. Loss: 0.5398815870285034. Accuracy: 65.18218623481782%
Iteration: 800. Loss: 0.8142744302749634. Accuracy: 65.9919028340081%
Iteration: 900. Loss: 0.18974590301513672. Accuracy: 65.9919028340081%
EPOCH -- 1
Iteration: 0. Loss: 0.47977590560913086. Accuracy: 65.58704453441295%
Iteration: 100. Loss: 0.08942508697509766. Accuracy: 65.58704453441295%
Iteration: 200. Loss: 1.9641937017440796. Accuracy: 55.06072874493927%
Iteration: 300. Loss: 0.9415032863616943. Accuracy: 65.99190

In [0]:
 def get_category_from(question):
  model_dict["Main"].eval()
  input_q, _ = feature_preparation(question)
  if torch.cuda.is_available():
    input_q = input_q.cuda()
  output = model_dict["Main"](input_q)[0]
  _, pred_label = torch.max(output.data, 1)
  prediction=list(main_category_labels.keys())[pred_label]
  return prediction

In [66]:
import random
for i in random.sample(range(0, category_dataset.shape[0]), 10):
  print("Question index: ", i)
  print(category_dataset["question"][i])
  print("Actual Category label:", category_dataset.label[i])
  print("Predicted Category label: ", get_category_from(category_dataset.question[i]))
  print("")

Question index:  169
Where are root hair cell close too?
Actual Category label: Cells and Organisms
Predicted Category label:  Cells and Organisms

Question index:  232

Actual Category label: Cells and Organisms
Predicted Category label:  Cells and Organisms

Question index:  16
What kind of images do a scanning electron microscope give?
Actual Category label: Cells and Organisms
Predicted Category label:  Cells and Organisms

Question index:  317
Why do people not realise that they have HIV?
Actual Category label: Cells and Organisms
Predicted Category label:  Cells and Organisms

Question index:  634

Actual Category label: Disease and bioenergetics
Predicted Category label:  Cells and Organisms

Question index:  707
Why does the stomach produce acid?
Actual Category label: Disease and bioenergetics
Predicted Category label:  Cells and Organisms

Question index:  1172
 Explain why fossils are so valuable to scientists ?
Actual Category label: Genetics and reproduction
Predicted Cate

In [47]:
# Train Section-Classifier Model
# loss_function = nn.CrossEntropyLoss()
learning_rate = 1e-05
section_model_optimizer = optim.Adam(params =  model_dict["Sections"].parameters(), lr=learning_rate)
train_on(model_dict["Sections"], training_loaders["Sections"], testing_loaders["Sections"], section_model_optimizer)
# Save and clear model 
torch.save(model_dict["Sections"].state_dict(), 'drive/My Drive/Layer 1 Models/roberta_sections_model.pth')
model_dict["Sections"] = None

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


EPOCH -- 0
Iteration: 0. Loss: 2.393050193786621. Accuracy: 6.086956521739131%
Iteration: 100. Loss: 1.8546733856201172. Accuracy: 11.73913043478261%
Iteration: 200. Loss: 2.248194694519043. Accuracy: 13.478260869565217%
Iteration: 300. Loss: 2.6219496726989746. Accuracy: 22.17391304347826%
Iteration: 400. Loss: 2.8369686603546143. Accuracy: 23.47826086956522%
Iteration: 500. Loss: 1.432145118713379. Accuracy: 23.91304347826087%
Iteration: 600. Loss: 2.1862945556640625. Accuracy: 23.91304347826087%
Iteration: 700. Loss: 2.404372453689575. Accuracy: 23.47826086956522%
Iteration: 800. Loss: 3.017977237701416. Accuracy: 30.0%
Iteration: 900. Loss: 1.3736419677734375. Accuracy: 33.91304347826087%
EPOCH -- 1
Iteration: 0. Loss: 2.192647933959961. Accuracy: 33.47826086956522%
Iteration: 100. Loss: 1.5098868608474731. Accuracy: 30.869565217391305%
Iteration: 200. Loss: 0.5173544883728027. Accuracy: 34.78260869565217%
Iteration: 300. Loss: 1.5833520889282227. Accuracy: 37.391304347826086%
Iter

In [0]:
 def get_section_from(question):
  model_dict["Sections"].eval()
  input_q, _ = feature_preparation(question)
  if torch.cuda.is_available():
    input_q = input_q.cuda()
  output = model_dict["Sections"](input_q)[0]
  _, pred_label = torch.max(output.data, 1)
  prediction=list(section_labels.keys())[pred_label]
  return prediction

In [91]:
# Trial the section-classifier
for i in random.sample(range(0, section_dataset.shape[0]), 10):
  if((get_section_from(section_dataset.question[i]) == section_dataset.label[i]) and (section_dataset["question"][i])):
    print(section_dataset["question"][i])
    print("Actual Category label:", section_dataset.label[i])
    print("Predicted Category label: ", get_section_from(section_dataset.question[i]))
    print("")

 Who do you always inherit your mitochondrial DNA from?
Actual Category label: Reproduction
Predicted Category label:  Reproduction



In [125]:
# Train Full Sub-section-Classifier Model
# loss_function = nn.CrossEntropyLoss()
learning_rate = 1e-04
section_model_optimizer = optim.Adam(params =  model_dict["Full Sub-sections"].parameters(), lr=learning_rate)
train_on(model_dict["Full Sub-sections"], training_loaders["Full Sub-sections"], testing_loaders["Full Sub-sections"], section_model_optimizer)
# Save and clear model 
torch.save(model_dict["Full Sub-sections"].state_dict(), 'drive/My Drive/Layer 1 Models/roberta_full_subsections_model.pth')
# model_dict["Full Sub-sections"] = None

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


EPOCH -- 0
Iteration: 0. Loss: 3.5625808238983154. Accuracy: 0.4048582995951417%
Iteration: 100. Loss: 2.842125415802002. Accuracy: 0.0%
Iteration: 200. Loss: 5.295422077178955. Accuracy: 2.42914979757085%
Iteration: 300. Loss: 4.225673675537109. Accuracy: 4.8582995951417%
Iteration: 400. Loss: 4.184029579162598. Accuracy: 0.0%
Iteration: 500. Loss: 6.064929962158203. Accuracy: 0.0%
Iteration: 600. Loss: 4.6203179359436035. Accuracy: 0.0%
Iteration: 700. Loss: 4.219280242919922. Accuracy: 6.0728744939271255%
Iteration: 800. Loss: 3.5468759536743164. Accuracy: 0.0%
Iteration: 900. Loss: 5.635259628295898. Accuracy: 0.8097165991902834%
EPOCH -- 1
Iteration: 0. Loss: 3.36755633354187. Accuracy: 1.214574898785425%
Iteration: 100. Loss: 3.667219400405884. Accuracy: 0.0%
Iteration: 200. Loss: 5.015081405639648. Accuracy: 0.0%
Iteration: 300. Loss: 5.256465435028076. Accuracy: 0.0%
Iteration: 400. Loss: 4.811500549316406. Accuracy: 0.0%
Iteration: 500. Loss: 4.125120639801025. Accuracy: 3.643

In [0]:
# Trial the full sub-section classifier
for i in range(len(section_dataset.question)):
  if((get_section_from(section_dataset.question[i]) == section_dataset.label[i]) and (section_dataset["question"][i])):
    print(section_dataset["question"][i])
    print("Actual Category label:", section_dataset.label[i])
    print("Predicted Category label: ", get_section_from(section_dataset.question[i]))
    print("")
    continue

In [44]:
# Train Subsection-Classifier Model
# loss_function = nn.CrossEntropyLoss()
learning_rate = 1e-05

for section, model in model_dict["Subsections"].items():
  print("BEGIN TRAINING FOR SECTION: {} ------------------------------ ".format(section))
  print("")
  subsection_model_optimizer = optim.Adam(params =  model.parameters(), lr=learning_rate)
  train_on(model, training_loaders["Subsections"][section], testing_loaders["Subsections"][section], subsection_model_optimizer)
  # Save and clear model
  torch.save(model.state_dict(), 'drive/My Drive/Layer 1 Models/roberta_for_section_' + section.replace(" ", "_") +'_model.pth')
  model = None
  print("END OF TRAINING FOR SECTION: {} ------------------------------ ".format(section))
  print("")

BEGIN TRAINING FOR SECTION: Cell Strucuture and transport ------------------------------ 



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


EPOCH -- 0
Iteration: 0. Loss: 2.3003311157226562. Accuracy: 1.8181818181818181%
Iteration: 50. Loss: 1.9503686428070068. Accuracy: 0.0%
Iteration: 100. Loss: 2.2939772605895996. Accuracy: 0.0%
Iteration: 150. Loss: 3.1571922302246094. Accuracy: 0.0%
Iteration: 200. Loss: 2.407602310180664. Accuracy: 0.0%
EPOCH -- 1
Iteration: 0. Loss: 2.5395772457122803. Accuracy: 0.0%
Iteration: 50. Loss: 2.3547816276550293. Accuracy: 0.0%
Iteration: 100. Loss: 2.2929165363311768. Accuracy: 0.0%
Iteration: 150. Loss: 2.301271677017212. Accuracy: 0.0%
Iteration: 200. Loss: 2.0845162868499756. Accuracy: 0.0%
EPOCH -- 2
Iteration: 0. Loss: 2.6856472492218018. Accuracy: 0.0%
Iteration: 50. Loss: 2.452244758605957. Accuracy: 0.0%
Iteration: 100. Loss: 1.6746504306793213. Accuracy: 0.0%
Iteration: 150. Loss: 2.1499319076538086. Accuracy: 0.0%
Iteration: 200. Loss: 1.7714674472808838. Accuracy: 0.0%

END OF TRAINING FOR SECTION: Cell Strucuture and transport ------------------------------ 

BEGIN TRAINING F

EPOCH -- 0
Iteration: 0. Loss: 1.875667691230774. Accuracy: 0.0%
EPOCH -- 1
Iteration: 0. Loss: 0.7370411157608032. Accuracy: 46.15384615384615%
EPOCH -- 2
Iteration: 0. Loss: 1.3731069564819336. Accuracy: 53.84615384615385%

END OF TRAINING FOR SECTION: Cell Division ------------------------------ 

BEGIN TRAINING FOR SECTION: Organisation and the digestive system ------------------------------ 



EPOCH -- 0
Iteration: 0. Loss: 1.597841739654541. Accuracy: 0.0%
Iteration: 50. Loss: 2.226827383041382. Accuracy: 0.0%
EPOCH -- 1
Iteration: 0. Loss: 1.6401197910308838. Accuracy: 35.294117647058826%
Iteration: 50. Loss: 2.483398675918579. Accuracy: 0.0%
EPOCH -- 2
Iteration: 0. Loss: 2.34710955619812. Accuracy: 11.764705882352942%
Iteration: 50. Loss: 2.8535962104797363. Accuracy: 70.58823529411765%

END OF TRAINING FOR SECTION: Organisation and the digestive system ------------------------------ 

BEGIN TRAINING FOR SECTION: Organising Animals and Plants ------------------------------ 



...

EPOCH -- 0
Iteration: 0. Loss: 0.9702850580215454. Accuracy: 0.0%
EPOCH -- 1
Iteration: 0. Loss: 1.0783923864364624. Accuracy: 100.0%
EPOCH -- 2
Iteration: 0. Loss: 1.3607728481292725. Accuracy: 100.0%

END OF TRAINING FOR SECTION: Photosynthesis ------------------------------ 

BEGIN TRAINING FOR SECTION: Respiration ------------------------------ 



EPOCH -- 0
Iteration: 0. Loss: 1.2065050601959229. Accuracy: 0.0%
EPOCH -- 1
Iteration: 0. Loss: 0.5253492593765259. Accuracy: 100.0%
EPOCH -- 2
Iteration: 0. Loss: 1.060059666633606. Accuracy: 0.0%

END OF TRAINING FOR SECTION: Respiration ------------------------------ 

BEGIN TRAINING FOR SECTION: The human nervous system ------------------------------ 



EPOCH -- 0
Iteration: 0. Loss: 1.8985050916671753. Accuracy: 0.0%
Iteration: 50. Loss: 1.1178340911865234. Accuracy: 18.75%
EPOCH -- 1
Iteration: 0. Loss: 0.8917025923728943. Accuracy: 0.0%
Iteration: 50. Loss: 1.5273773670196533. Accuracy: 18.75%
EPOCH -- 2
Iteration: 0. Loss: 1.3480840921401978. Accuracy: 100.0%
Iteration: 50. Loss: 0.7877321839332581. Accuracy: 100.0%

END OF TRAINING FOR SECTION: The human nervous system ------------------------------ 

BEGIN TRAINING FOR SECTION: Reproduction ------------------------------ 



EPOCH -- 0
Iteration: 0. Loss: 2.176201343536377. Accuracy: 100.0%
Iteration: 50. Loss: 1.5505099296569824. Accuracy: 26.666666666666668%
EPOCH -- 1
Iteration: 0. Loss: 1.8101609945297241. Accuracy: 0.0%
Iteration: 50. Loss: 1.9678782224655151. Accuracy: 0.0%
EPOCH -- 2
Iteration: 0. Loss: 1.4182939529418945. Accuracy: 6.666666666666667%
Iteration: 50. Loss: 1.7344194650650024. Accuracy: 100.0%

END OF TRAINING FOR SECTION: Reproduction ------------------------------ 

BEGIN TRAINING FOR SECTION: Genetics and evolution ------------------------------ 



EPOCH -- 0
Iteration: 0. Loss: 1.7768151760101318. Accuracy: 0.0%
Iteration: 50. Loss: 2.0314760208129883. Accuracy: 22.22222222222222%
Iteration: 100. Loss: 2.322873830795288. Accuracy: 51.851851851851855%
EPOCH -- 1
Iteration: 0. Loss: 1.5824335813522339. Accuracy: 59.25925925925926%
Iteration: 50. Loss: 1.7917852401733398. Accuracy: 11.11111111111111%
Iteration: 100. Loss: 1.768058180809021. Accuracy: 18.51851851851852%
EPOCH -- 2
Iteration: 0. Loss: 1.677393913269043. Accuracy: 0.0%
Iteration: 50. Loss: 2.708089590072632. Accuracy: 62.96296296296296%
Iteration: 100. Loss: 1.7494465112686157. Accuracy: 44.44444444444444%

END OF TRAINING FOR SECTION: Genetics and evolution ------------------------------ 



In [45]:
!ls drive/'My Drive'/Layer\ 1\ Models

Final_Test_Set.csv
roberta_for_section_Cell_Division_model.pth
roberta_for_section_Cell_Strucuture_and_transport_model.pth
roberta_for_section_Communicable_diseases_model.pth
roberta_for_section_Genetics_and_evolution_model.pth
roberta_for_section_Non-Communicable_diseases_model.pth
roberta_for_section_Organisation_and_the_digestive_system_model.pth
roberta_for_section_Organising_Animals_and_Plants_model.pth
roberta_for_section_Photosynthesis_model.pth
roberta_for_section_Preventing_and_treating_disease_model.pth
roberta_for_section_Reproduction_model.pth
roberta_for_section_Respiration_model.pth
roberta_for_section_The_human_nervous_system_model.pth
roberta_full_subsections_model.pth
roberta_main_model.pth
roberta_sections_model.pth
test_dataset.csv
train_dataset.csv


In [0]:
 def get_subsection_from(question, section):
  input_q, _ = feature_preparation(question)
  if torch.cuda.is_available():
    input_q = input_q.cuda()

  model_dict["Subsections"][section].eval()
  sub_out = model_dict["Subsections"][section](input_q)[0]
  _, pred_label = torch.max(sub_out.data, 1)
  subsection = list(subsection_labels[section].keys())[pred_label]
  return subsection

In [75]:
ix = random.randint(0, len(subsection_datasets_train["Cell Division"]))

print(subsection_datasets_train["Cell Division"].question[ix])
print("Actual subsection label:", subsection_datasets_train["Cell Division"].label[ix])
print("Predicted label: ", get_subsection_from(subsection_datasets_train["Cell Division"].question[ix], "Cell Division"))
print("")

Explain the main arguments for and against the use of embryonic stem cells in medical research ?
Actual subsection label: Stem cell dilemmas
ERROR! Session/line number was not unique in database. History logging moved to new session 63
Predicted label:  Stem cells



In [0]:
def get_passage_for(subsection):
  return passage_df.loc[passage_df['Sub-section'] == subsection, ['Text']].values[0][0]

In [0]:
# Make drive paths for models:
category_model_path = 'drive/My Drive/Layer 1 Models/roberta_main_model.pth'
section_model_path = 'drive/My Drive/Layer 1 Models/roberta_sections_model.pth'
full_subsection_model_path = 'drive/My Drive/Layer 1 Models/roberta_full_subsections_model.pth'
subsection_model_paths = { }
for section, _ in subsection_labels.items():
  subsection_model_paths[section] = 'drive/My Drive/Layer 1 Models/roberta_for_section_' + section.replace(" ", "_") +'_model.pth'


def layer_1(question):
  input_question, _ = feature_preparation(question)
  if torch.cuda.is_available():
    input_question = input_question.cuda()

  # Get subsection from question
  model_dict["Sections"].load_state_dict(torch.load(section_model_path, map_location=device))
  section = get_section_from(question)
  print("Got section: ", section)

  # Get paragraph from subsection:
  model_dict["Subsections"][section].load_state_dict(torch.load(subsection_model_paths[section], map_location=device))
  subsection = get_subsection_from(question, section)
  print("Got subsection: ", subsection)
  final_passage = get_passage_for(subsection)
  return final_passage

In [88]:
print(section_dataset.question[400])
predicted_passage = layer_1(section_dataset.question[400])
print("Got passage:")
print(predicted_passage)

How does the small intestine increase diffusion and active transport into the blood?
Got section:  Reproduction
Got subsection:  Gene expression and mutation
Got passage:
Our understanding of how the genetic code in our cells produces organisms we see around us is growing all the time. Not only do we understand what happens when things go wrong as the DNA in our cells replicate. We are also beginning to understand how genes are switched on and off in our cells as we grow and develop. It seems as if our environment may affect exactly which genes come into use. When a gene codes for a protein that is synthesised in the cell, the gene is said to be expressed. Most of the DNA does not actually code for proteins, remember that there are only 21 000 genes but 3 billion pairs of bases! Scientists are still discovering what the non-coding part of the DNA does. One thing they have discovered is that the non-coding pans of the DNA are involved in switching genes, or parts of genes, on and off.Th

  # LAYER 2: Untouched as of yet.

In [49]:
from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample

from transformers.data.metrics.squad_metrics import compute_predictions_logits

model_name_or_path = "ktrapeznikov/albert-xlarge-v2-squad-v2"
# model_name_or_path = "bert-base-uncased"

output_dir = ""


def to_list(tensor):
    return tensor.detach().cpu().tolist()
    
# Config
n_best_size = 1
max_answer_length = 30
do_lower_case = True
null_score_diff_threshold = 0.0

# Setup model
config_class, model_class, tokenizer_class = (
    AlbertConfig, AlbertForQuestionAnswering, AlbertTokenizer)


config = config_class.from_pretrained(model_name_or_path)

tokenizer = tokenizer_class.from_pretrained(model_name_or_path, do_lower_case=True)

model = model_class.from_pretrained(model_name_or_path, config=config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

processor = SquadV2Processor()

def run_prediction(question_texts, context_text):
    """Setup function to compute predictions"""
    examples = []

    for i, question_text in enumerate(question_texts):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            is_impossible=False,
            answers=None,
        )

        examples.append(example)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=384,
        doc_stride=128,
        max_query_length=64,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    all_results = []

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)

    output_prediction_file = "predictions.json"
    output_nbest_file = "nbest_predictions.json"
    output_null_log_odds_file = "null_predictions.json"

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        n_best_size,
        max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,  # verbose_logging
        True,  # version_2_with_negative
        null_score_diff_threshold,
        tokenizer,
    )

    return predictions

In [0]:
context = "Animals need to eat food to get their energy. But green plants and algae do not. Instead they make their own food in a process called photosynthesis. Almost all life on Earth depends upon this process. Photosynthesis is also important in maintaining the levels of oxygen and carbon dioxide in the atmosphere. These are the things that plants need for photosynthesis: carbon dioxide, water, light (a source of energy) These are the things that plants make by photosynthesis: glucose and oxygen Photosynthesis takes place inside plant cells in small objects called chloroplasts. Chloroplasts contain a green substance called chlorophyll. This absorbs the light energy needed to make photosynthesis happen. Plants and algae can only carry out photosynthesis in the light. Plants get carbon dioxide from the air through their leaves, and water from the ground through their roots. Light energy comes from the Sun. The oxygen produced is released into the air from the leaves. The glucose produced can be turned into other substances, such as starch and plant oils, which are used as an energy store. This energy can be released by respiration."
questions = ["What are the products of photosynthesis?", 
             "What colour is chlorophyll?",
             "Where does photosynthesis take place?",
             "When do plants photosynthesise?",
             "What are the reactants of photosynthesis?"]

# Run method
predictions = run_prediction(questions, context)

# Print results
for key in predictions.keys():
  print(predictions[key])

In [0]:
test_dataset_url = "drive/My Drive/Layer 1 Models/Final_Test_Set.csv"
final_test_df = pd.read_csv(test_dataset_url)

In [57]:
final_test_df["Question"].values

array(['Define hypotonic solution ?',
       'What is the disease with symptoms of thick, sticky mucus in their lungs, gut and reproductive systems?',
       'What health problems can cystic fibrosis cause?',
       'Why does having cystic fibrosis cause health problems?',
       'What have scientists shown is linked to the rate of cell respiration?',
       'What makes up the cytoplasm?',
       'What happens to the cytoplasm if itâ€™s cell uses up all its water in chemical reactions?',
       'How does the human body ventilate the alveoli?',
       'What are the air sacs in the lungs called?', 'What are alveoli?',
       'Why are the alveoli effective for gas exchange?',
       'What is an adult stem cell?',
       'What is a good source of adult stem cells?',
       ' Why is it important to restrict the agricultural use of antibiotics ?',
       ' Why are some bacteria in hospitals resistant to antibiotics ?',
       ' What should medical staff do to prevent the spread of these muta

ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [0]:
def answer_it_BERT(question):
  passage = layer_1(question)
  predictions = run_prediction([question], passage)
  predictions

In [93]:
prediction = answer_it_BERT("Who do you always inherit your mitochondrial DNA from?")
print("")
print("QUESTION: Who do you always inherit your mitochondrial DNA from?")
print("BERT ANSWERED: ")
print(prediction)

Got section:  Reproduction
Got subsection:  Gene expression and mutation


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3663.15it/s]



QUESTION: Who do you always inherit your mitochondrial DNA from?
BERT ANSWERED:  


In [0]:
import time

In [68]:
start = time.process_time()
for question in final_test_df["Question"]:
  answer = answer_it_BERT(question)
  final_test_df.loc[final_test_df["Question"] == question, "Answer"] = answer

final_test_df.head(10)

print(time.process_time() - start)

Got section:  Genetics and evolution
Got subsection:  Classification


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1998.24it/s]


Got section:  Preventing and treating disease
Got subsection:  Discovering and developing drugs


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1834.78it/s]


Got section:  Genetics and evolution
Got subsection:  Fossils and extinction 


...

Got section:  Preventing and treating disease
Got subsection:  Discovering and developing drugs


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2511.56it/s]


Got section:  Genetics and evolution
Got subsection:  Classification


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 4809.98it/s]


83.853301905


In [0]:
final_test_df.to_csv("drive/My Drive/Layer 1 Models/Final_Test_Set_answered.csv")